# 데이터 크롤링

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

baseUrl = 'http://www.10000recipe.com/recipe/'
                    
def CrawlingBetweenRanges(startRecipeId, endRecipeId):
    recipe_data_list = []

    for i in range(startRecipeId, endRecipeId):
        res = PageCrawler(str(i))
        time.sleep(random.randint(3, 5))
        if res:
            recipe_data_list.append(res)
    
    return recipe_data_list


def PageCrawler(recipeUrl):
    url = baseUrl + recipeUrl

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    recipe_title = []  # 레시피 제목
    recipe_source = {}  # 레시피 재료
    recipe_step = []  # 레시피 순서

    try:
        res = soup.find('div', 'view2_summary')
        res = res.find('h3')
        recipe_title.append(res.get_text())
        res = soup.find('div', 'view2_summary_info')
        recipe_title.append(res.get_text().replace('\n', ''))
        res = soup.find('div', 'ready_ingre3')
    except(AttributeError):
        return

    try:
        for n in res.find_all('ul'):
            source = []
            title = n.find('b').get_text()
            recipe_source[title] = ''
            for tmp in n.find_all('li'):
                tempSource = tmp.get_text().replace('\n', '').replace(' ', ' ')
                source.append(tempSource.split("    ")[0])
                source.append(tempSource.split("    ")[-1])

            recipe_source[title] = source
    except (AttributeError):
        return

    res = soup.find('div', 'view_step')
    i = 0
    for n in res.find_all('div', 'view_step_cont'):
        i = i + 1
        recipe_step.append('#' + str(i) + ' ' + n.get_text().replace('\n', ' '))

    img_tag = soup.find('img', {'id': 'main_thumbs'})
    if img_tag:
        recipe_image_url = img_tag['src']

    if not recipe_step:
        return

    recipe_all = [recipe_title, recipe_source, recipe_step, recipe_image_url]
    return recipe_all

In [98]:
# df = CrawlingBetweenRanges(7000000,7000300)
df = CrawlingBetweenRanges(7002001,7002300)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [99]:
df

[[['아이반찬 소고기 떡갈비 레시피 떡갈비소스 만들기', '2인분 30분 이내 초급 '],
  {'[재료]': ['다진소고기',
    '구매350g',
    '다진돼지고기',
    '구매100g',
    '찹쌀가루',
    '구매2T',
    '잣',
    '구매1줌'],
   '[고기양념]': ['간장',
    '구매3T',
    '황설탕',
    '구매2T',
    '매실액',
    '구매1T',
    '꿀',
    '구매1/2T',
    '다진마늘',
    '구매1T',
    '다진대파',
    '구매2T',
    '잣',
    '구매1T',
    '참기름',
    '구매1.5T',
    '후추',
    '구매약간'],
   '[소스]': ['물', '구매3T', '간장', '구매1T', '꿀', '구매1T', '참기름', '구매1t']},
  ['#1 고기 양념을 만들어 줍니다.간장 3 큰 술, 황설탕 2 큰 술 꿀 1/2 큰 술, 매실액 1 큰 술 다진 마늘 1 큰 술, 다진 대파 2 큰 술 참기름 1½ 큰 술, 잣가루 1 큰 술 후추 약간잣가루를 같이 넣어주면고소한 풍미가 더 깊어 조금 더 고급스러운 맛을 낼 수 있어요.',
   '#2 고기에 찹쌀가루 1큰술을 반죽에 같이 넣어준 뒤 간이 고르게 어우러지도록 치대어 줍니다. ',
   '#3 만든 고기 반죽을 동그랗게 빚어줍니다.',
   '#4 달군 팬에 식용유 코팅해 준 뒤 앞뒤로 노릇노릇하게 구워줍니다. 먼저 중간 불로 겉면을 익혀준 뒤 약불로 낮추어 속까지 구워줍니다.',
   '#5 물 3큰술, 간장 1큰술, 꿀 1큰술, 참기름 1작은술 넣고 잘 섞어 소스를 만들어 줍니다.',
   '#6 2/3 이상 익으면 소스를 고르게 발라 구워줍니다.',
   '#7 그릇에 먹음직스럽게 담아줍니다. 취향에 따라 잣가루를 올려주시면 고소하게 잘 어울리고 전통의 고급스러움을 맛보실 수 있어요.'],
  'https://recipe1.ezmember.co.kr/

In [7]:
# df[0] # 제목
# df[1] # 전처리 필요 재료,소스
# df[2] # 레시피
# df[3] # 이미지 주소

# 데이터 전처리

In [100]:
final_result = []

for element in df:
    is_first = True

    title = element[0][0]
    ingredients = element[1]
    recipe = element[2]
    image_url = element[3]

    result = {}
    ingredients_list = []
    source_list = []

    if len(ingredients) != 2:
        continue

    for ingredients_string in ingredients:
        temp = ingredients[ingredients_string]
        temp_list = []
        for i in range(0, len(temp), 2):
            if is_first:
                ingredients_list.append(temp[i] + ' ' + temp[i+1].replace('구매', ''))
            else:
                source_list.append(temp[i] + ' ' + temp[i+1].replace('구매', ''))
        is_first = False

    result['title'] = title
    result['ingredients'] = ingredients_list
    result['source'] = source_list
    result['recipe'] = recipe
    result['image_url'] = image_url
    
    final_result.append(result)

In [101]:
final_result

[{'title': '소고기알배춧국',
  'ingredients': ['소고기 200g', '알배추 300g', '콩나물 300g', '대파 1대'],
  'source': ['참기름 1큰술',
   '큐브 생강 1개',
   '고춧가루 3큰술',
   '생수 2L',
   '간마늘 2큰술',
   '어간장 2큰술',
   '국 간장 1큰술',
   '소금 1작은술',
   '후추 톡톡'],
  'recipe': ['#1 냄비에 참기름 1 큰 술을 두르고, 소고기 양지 200g 을 넣어 줍니다.',
   '#2 소고기가 옅은 회색이 나도록 볶아 줍니다',
   '#3 소고기가 익어 핏물이 안 보이면 고춧가루 3 큰 술을 넣고 볶아 줍니다.고추 가루를 넣고 볶으면 고추기름 효과가 나서 국물 색이 예쁘게 붉게 어우러집니다.',
   '#4 생수 2L 을 붓고, 큐브 생강 1개도 넣은 후 푹 끓여 줍니다.생강을 넣고 푹 끓이면 고기가 빨리 연해집니다.',
   '#5 콩나물 1봉지를 세척해 줍니다',
   '#6 알 배추를 세척한 후 그릇에 생수 1컵을 담고 전자레인지에서 2분 돌려 데쳐 준 다음 먹기 좋게 찢어 줍니다.',
   '#7 대파 1대는 어슷썰기 해 줍니다.',
   '#8 냄비에 콩나물, 알 배추를 넣고 국이 끓으면 어 간장 2 큰 술과 국 간장 1 큰 술, 간 마늘 2큰 술 을 넣고 끓여 줍니다.',
   '#9 소금 1작은 술 넣고 간을 맞추고 먹을 때 후추를 넣어 먹으면 됩니다.',
   '#10 푹 끓인 후 국 간을 보고 대파를 넣어 줍니다.',
   '#11 그릇에 옮겨 담고 후추도 톡톡 넣어 줍니다.'],
  'image_url': 'https://recipe1.ezmember.co.kr/cache/recipe/2023/05/10/6050279e49058b411d195e3395d027cd1.jpg'},
 {'title': '국민 밑반찬 마른 멸치 볶음 만드는법',
  'ingredients': ['잔멸치 2줌',
   '꼴뚜기 1줌',
 

# 데이터 베이스 Input

In [102]:
import pymysql
import requests

# MariaDB에 연결
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
    database="recipe"
)

def download_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()

        image_data = response.content

        return image_data
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None
    
for element in final_result:
# 커서 생성
    mycursor = mydb.cursor()

    image_data = download_image_from_url(element['image_url'])
    
    # 데이터 삽입 쿼리  
    sql = "INSERT INTO new_content (title, ingredients, source, recipe, image) VALUES (%s, %s, %s, %s, %s)"
    val = (str(element['title']), str(element['ingredients']), str(element['source']), str(element['recipe']), image_data)  # 실제 데이터 값
 
    try:
        # 데이터베이스에 쿼리 실행
        mycursor.execute(sql, val)

        # 변경사항 커밋
        mydb.commit()

        print(mycursor.rowcount, "record inserted.")  # 삽입된 레코드 수 출력

    except Exception as e:
        # 에러 발생 시 롤백
        print("Error occurred:", e)
        mydb.rollback()

# 연결 종료
mydb.close()

1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record ins

# 결과 확인용

In [2]:
import pymysql

mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
    database="recipe"
)

# MariaDB 연결
cursor = mydb.cursor()

sql = 'SELECT recipe FROM new_content'

ingredients_list = []

try:
    cursor.execute(sql)
    # fetchall()을 사용하여 결과를 리스트로 가져옴
    results = cursor.fetchall()
    
    for row in results:
        ingredients_list.append(row)

except Exception as e:
    # 에러 발생 시 롤백
    print("Error occurred:", e)

finally:
    mydb.close()

# 결과 출력
for ingredients in ingredients_list:
    print(ingredients)

("['#1 잘익은 배추김치(묵은지)를 먹기좋은크기로 잘라서 냄비에 담아주세요', '#2 베이키드빈스 콩통조림을 준비후에 채반에 넣어 찬물에 샤워시켜주세요 그냥 다넣으시면 김치찌개국물이 걸죽해지네요', '#3 스팸을 한입크기로 잘라주세요', '#4 청양고추와 대파를 쫑쫑썰어서 준비해주세요', '#5 썰어준 배추김치에 살짝 잠길정도로 물이나 다시멸치육수등을 부어주신후 센불에서 펄펄 끓여주시다가', '#6 자른스팸,청양고추,대파를 넣어주신후 다진마늘을 넣어서 펄펄 15~20분간 끓여주세요 배추김치의 새콤함에 따라서 소량의 설탕을 넣어주세요 끓인후 국물의 간을 보시고 싱거우시면 소금을 넣어주세요 *고추가루를 조금 추가해주셔도 되세요', '#7 20분정도 펄펄 끓여주신후에 씻어놓은 베이키드빈스콩을 넣어주신후 3~5분을 더 끓여주시면 되세요', '#8 부대찌개인듯~스팸김치찌개인듯~간단하게 끓여서 먹기좋은 쉬운찌개입니다 요즘처럼 캠핑하기좋은때에 재료를 다썰어 통에 담아가셨다가 조리해서 드시면 술안주로 또는 밥에 비벼드시기에 좋아요']",)
("['#1 재료준비  가지를 비롯한 모든 채소들 씻어 먹기좋은 크기로 잘라 준비', '#2 기름 두른 팬에 대파 넣고 센 불에서 볶아 대파 기름 만들고', '#3 돼지고기 다짐 육 넣고 다진 마늘 반 큰술과 맛술, 후추 조금 넣고 볶아요다짐육은 미리 밑간해두었다가 사용해도 좋아요', '#4 다짐육이 어느 정도 익어가면 준비한 채소들 넣고 볶아주고', '#5 가지 넣고 3~5분 정도 볶아요 ', '#6 물 한 컵 넣고 바글바글 끓여주고 덮밥양념 넣어서 잘 섞어주며 볶다가 약불로 줄이고', '#7 전분 물 조금씩 넣어가며 농도 맞춰줍니다', '#8 깨 송송뿌려 가지덮밥 소스 완성', '#9 볼에 밥 담아주고 만든 가지덮밥소스 담아냅니다 참기름 톡 한번 더 하면 고소함 두배~']",)
("['#1 볼에 송송 썬 대파와 오이고추, 사방 1cm으로 썰은 토마토를 넣어줍니다.대파 10cm, 오이고추 4개, 토마토 1개', '#2 다진 마늘 1큰

# 이미지 확인용코드

In [1]:
import pymysql

from io import BytesIO
from PIL import Image

# MariaDB 연결 설정
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
    database="recipe"
)

# 커서 생성
cursor = mydb.cursor()

# 이미지 데이터를 가져올 쿼리 작성
select_query = "SELECT title, image FROM new_content"

# 쿼리 실행
cursor.execute(select_query)

# 결과 가져오기
result = cursor.fetchone()
print(result[0])
# 이미지 데이터 읽기
if result:
    image_data = result[1]

    # Pillow를 사용하여 이미지 표시
    image = Image.open(BytesIO(image_data))
    image.show()

# 연결 닫기
cursor.close()
mydb.close()

베이키드빈스콩(부대찌개콩)스팸김치찌개!
